## 기업 데이터에 추가 데이터 붙이기

In [23]:
import pandas as pd

gi = pd.read_csv('../data/gi.csv')

In [24]:
gi = gi.iloc[:,1:]

In [25]:
gi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   회사명         533 non-null    object
 1   stock_code  533 non-null    int64 
 2   상장일         533 non-null    object
dtypes: int64(1), object(2)
memory usage: 12.6+ KB


In [26]:
gi

,회사명,stock_code,상장일
0,바이오노트,377740,2022-12-22
1,쏘카,403550,2022-08-22
2,수산인더스트리,126720,2022-08-01
3,LG에너지솔루션,373220,2022-01-27
4,카카오페이,377300,2021-11-03
...,...,...,...
528,싸이맥스,160980,2015-06-17
529,픽셀플러스,87600,2015-06-12
530,제노포커스,187420,2015-05-29
531,유지인트,195990,2015-04-13


In [27]:
# 1) KOSPI200 데이터 읽기 & 수익률 계산
ko200 = pd.read_csv('../notion_data/KOSPI200.csv', parse_dates=['날짜'])
ko200 = ko200.sort_values('날짜').reset_index(drop=True)
ko200['kospi200(-20)'] = ko200['종가'].shift(1) / ko200['종가'].shift(20) - 1

# 2) gi 데이터 읽기
gi = pd.read_csv('../data/gi.csv', parse_dates=['상장일'])
gi = gi.sort_values('상장일').reset_index(drop=True)

# 3) asof merge 로 상장일 기준 최근 거래일의 kospi200(-20) 가져오기
merged = pd.merge_asof(
    gi,
    ko200[['날짜', 'kospi200(-20)']].sort_values('날짜'),
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

# 4) 결과 확인
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)
0         532    포시에스      189690 2015-02-11      -0.053391
1          68  세화아이엠씨      145210 2015-03-19       0.049828
2          67   엔에스쇼핑      138250 2015-03-27       0.058506
3         531    유지인트      195990 2015-04-13      -0.016436
4         530   제노포커스      187420 2015-05-29       0.028846


In [28]:
nasdaq = pd.read_csv('../notion_data/Nasdaq.csv', parse_dates=['Price'])
nasdaq = nasdaq.sort_values('Price').reset_index(drop=True)
nasdaq['nasdaq(-20)'] = nasdaq['Close'].shift(1) / nasdaq['Close'].shift(20) - 1

merged = pd.merge_asof(
    merged,
    nasdaq[['Price', 'nasdaq(-20)']].sort_values('Price'),
    left_on='상장일',
    right_on='Price',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['Price'])

# 4) 결과 확인
print(merged.head())



   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683


In [29]:

putcall = pd.read_csv('../notion_data/PutCall_ratio.csv',encoding='cp949',parse_dates=['일자'])
putcall = putcall.sort_values('일자').reset_index(drop=True)
putcall['putcall(-20)'] = putcall['P/C Ratio'].shift(1) / putcall['P/C Ratio'].shift(20) - 1

merged = pd.merge_asof(
    merged,
    putcall[['일자', 'putcall(-20)']].sort_values('일자'),
    left_on='상장일',
    right_on='일자',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['일자'])

# 4) 결과 확인
print(merged.head())



   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  
0      0.056075  
1     -0.338983  
2     -0.042553  
3     -0.070707  
4      0.324324  


In [30]:
vixcls = pd.read_csv('../notion_data/VIXCLS(미국).csv')
vixcls = vixcls.sort_values('observation_date').reset_index(drop=True)

# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
vixcls['observation_date'] = pd.to_datetime(vixcls['observation_date'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
vixcls = vixcls.sort_values('observation_date').reset_index(drop=True)

# 3) Perform the merge_asof
merged = pd.merge_asof(
    merged,
    vixcls[['observation_date', 'VIXCLS']],
    left_on='상장일',
    right_on='observation_date',
    direction='backward'
).drop(columns=['observation_date'])

print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  
0      0.056075   16.96  
1     -0.338983   14.07  
2     -0.042553   15.07  
3     -0.070707   13.94  
4      0.324324   13.84  


In [31]:
us10 = pd.read_csv('../notion_data/미국10년물국채금리채권수익률.csv')
us10 = us10.sort_values('날짜').reset_index(drop=True)
us10['미국10년물국채금리수익률(-20)'] = us10['종가'].shift(1) / us10['종가'].shift(20) - 1

# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
us10['날짜'] = pd.to_datetime(us10['날짜'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
us10 = us10.sort_values('날짜').reset_index(drop=True)

merged = pd.merge_asof(
    merged,
    us10[['날짜', '미국10년물국채금리수익률(-20)']].sort_values('날짜'),
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

# 4) 결과 확인
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  미국10년물국채금리수익률(-20)  
0      0.056075   16.96            0.080323  
1     -0.338983   14.07           -0.096136  
2     -0.042553   15.07            0.000000  
3     -0.070707   13.94           -0.060270  
4      0.324324   13.84            0.010890  


In [32]:
market_d1 = pd.read_csv('../notion_data/시장금리국고채3년(일별).csv',parse_dates=['변환'])
market_d1 = market_d1.sort_values('변환').reset_index(drop=True)

# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
market_d1['변환'] = pd.to_datetime(market_d1['변환'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
market_d1 = market_d1.sort_values('변환').reset_index(drop=True)

# 3) Perform the merge_asof
merged = pd.merge_asof(
    merged,
    market_d1[['변환', '원자료']],
    left_on='상장일',
    right_on='변환',
    direction='backward'
).drop(columns=['변환'])

merged.rename(columns={'원자료':'시장금리국고채3년'})
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  미국10년물국채금리수익률(-20)    원자료  
0      0.056075   16.96            0.080323  2.051  
1     -0.338983   14.07           -0.096136  1.825  
2     -0.042553   15.07            0.000000  1.770  
3     -0.070707   13.94           -0.060270  1.732  
4      0.324324   13.84            0.010890  1.745  


In [33]:
merged

,Unnamed: 0,회사명,stock_code,상장일,kospi200(-20),nasdaq(-20),putcall(-20),VIXCLS,미국10년물국채금리수익률(-20),원자료
0,532,포시에스,189690,2015-02-11,-0.053391,0.027060,0.056075,16.96,0.080323,2.051
1,68,세화아이엠씨,145210,2015-03-19,0.049828,0.011804,-0.338983,14.07,-0.096136,1.825
2,67,엔에스쇼핑,138250,2015-03-27,0.058506,-0.020181,-0.042553,15.07,0.000000,1.770
3,531,유지인트,195990,2015-04-13,-0.016436,0.025498,-0.070707,13.94,-0.060270,1.732
4,530,제노포커스,187420,2015-05-29,0.028846,0.031683,0.324324,13.84,0.010890,1.745
...,...,...,...,...,...,...,...,...,...,...
528,73,기가비스,420770,2023-05-24,-0.020188,0.059548,-0.094118,20.03,0.073440,3.378
529,72,진영,285800,2023-06-01,0.041520,0.075670,-0.132653,15.65,0.061426,3.478
530,71,나라셀라,405920,2023-06-02,-0.049039,0.094814,-0.135135,14.60,0.026804,3.436
531,70,마녀공장,439090,2023-06-08,-0.047255,0.064881,0.056180,13.65,0.095293,3.543
